# Datumsberechnungen

**Referenz**: [Date and time data types and functions](https://learn.microsoft.com/en-us/sql/t-sql/functions/date-and-time-data-types-and-functions-transact-sql?view=sql-server-ver16)

Folgende Funktionen sollten Sie in Ihr Repertoire auf jeden Fall aufnehmen:

- GETDATE()
- DATEDIFF()
- DAY()
- MONTH()
- YEAR()
- DATEFROMPARTS()
- EOMONTH()

## Übungen:

- Wie viele Tage sind es bis zum "nächsten ersten"?
- Wie viele Tage sind es bis zu Ihrem nächsten Geburtstag?

### Praxis mit der Tabelle **Adresse**:

Zu den Adressen sind Geburtsdaten angegeben.

Erstellen Sie eine Abfrage mit folgenden berechneten Spalten:

- Alter am heutigen Tag
- Alter zu Stichtag 31.12. d. J.
- Geburtstag in diesem Jahr
- nächster Geburtstag
- nächster runder Geburtstag

### Hinweis:

Die Funktion DATEDIFF verhält sich anders als die ähnliche Funktion in Excel: Sie berechnet die Differenz in Jahren nicht tagesgenau.

Für eine genaue Alterberechnung sind CASE-Anweisungen erforderlich.

In [ ]:
-- Diverse Datumsberechnungen
DECLARE @Datum DATE = '1994-04-15';
DECLARE @Heute DATE = CAST(GETDATE() AS DATE);
DECLARE @DatDiesJahr DATE = DATEFROMPARTS(YEAR(@Heute),MONTH(@Datum), DAY(@Datum));
DECLARE @NextDat DATE = DATEADD(YEAR, CASE WHEN @DatDiesJahr < @Heute THEN 1 ELSE 0 END , @DatDiesJahr);
DECLARE @Alter INT =  DATEDIFF(YEAR, @Datum, @Heute) + 
        CASE WHEN 
            (DATEADD(YEAR,DATEDIFF(YEAR, @Datum, @Heute) , @Datum) > @Heute) 
            THEN - 1 
            ELSE 0 
        END;
DECLARE @NextRundDat DATE = DATEADD(YEAR, 
        10 - (@Alter % 10)  +
        CASE 
            WHEN @DatDiesJahr < @Heute THEN 0 
            WHEN @DatDiesJahr =  @Heute THEN  0 
            ELSE  9
        END, @DatDiesJahr);

SELECT @Datum AS Datum, @DatDiesJahr AS [Datum dieses Jahr], @NextDat AS [Nächstes Datum], @Alter AS [Alter], @NextRundDat AS [Jubiläum];        

In [ ]:
CREATE OR ALTER FUNCTION dbo.udf_FullYearsSinceDate(@Datum AS DATE)
RETURNS INT
AS
BEGIN
    DECLARE @Heute DATE = CAST(GETDATE() AS DATE);
    DECLARE @Alter INT =  DATEDIFF(YEAR, @Datum, @Heute) + 
            CASE WHEN 
                (DATEADD(YEAR,DATEDIFF(YEAR, @Datum, @Heute) , @Datum) > @Heute) 
                THEN - 1 
                ELSE 0 
            END;
    RETURN @Alter;       
END;     

In [ ]:
CREATE OR ALTER FUNCTION dbo.udf_DateThisYear(@Datum AS DATE)
RETURNS DATE
AS
BEGIN
    DECLARE @Heute DATE = CAST(GETDATE() AS DATE);
    DECLARE @DatDiesJahr DATE = DATEFROMPARTS(YEAR(@Heute),MONTH(@Datum), DAY(@Datum));
    RETURN @DatDiesJahr;
END;            

In [ ]:
-- Nächster Geburtstag (ggf. nächstes Jahr)
CREATE OR ALTER FUNCTION dbo.udf_NextEvent(@Datum AS DATE)
RETURNS DATE
AS
BEGIN
    DECLARE @Heute DATE = CAST(GETDATE() AS DATE);
    DECLARE @DatDiesJahr DATE = DATEFROMPARTS(YEAR(@Heute),MONTH(@Datum), DAY(@Datum));
    DECLARE @NextDat DATE = DATEADD(YEAR, CASE WHEN @DatDiesJahr < @Heute THEN 1 ELSE 0 END , @DatDiesJahr);
    RETURN @NextDat;       
END;     

In [ ]:
-- Nächstes rundes Jubiläum
CREATE OR ALTER FUNCTION dbo.udf_NextRoundEvent(@Datum AS DATE)
RETURNS DATE
AS
BEGIN
    DECLARE @Heute DATE = CAST(GETDATE() AS DATE);
    DECLARE @DatDiesJahr DATE = DATEFROMPARTS(YEAR(@Heute),MONTH(@Datum), DAY(@Datum));
    DECLARE @Alter INT =  DATEDIFF(YEAR, @Datum, @Heute) + 
            CASE WHEN 
                (DATEADD(YEAR,DATEDIFF(YEAR, @Datum, @Heute) , @Datum) > @Heute) 
                THEN - 1 
                ELSE 0 
            END;
    DECLARE @NextRundDat DATE = DATEADD(YEAR, 
            10 - (@Alter % 10)  +
            CASE 
                WHEN @DatDiesJahr < @Heute THEN 0 
                WHEN @DatDiesJahr =  @Heute THEN  0 
                ELSE  9
            END, @DatDiesJahr);
     RETURN @NextRundDat;       
END;     

In [ ]:
-- Teste die Funktionen
SELECT AdressID
    , CONCAT(Vorname, ' ', Nachname) AS Person
    , Geburtsdatum
    , dbo.udf_FullYearsSinceDate(Geburtsdatum) AS [Alter]
    , dbo.udf_DateThisYear(Geburtsdatum) AS [Geburtstag dieses Jahr]
    , dbo.udf_NextEvent(Geburtsdatum) AS [Nächster Geburtstag]
    , dbo.udf_NextRoundEvent(Geburtsdatum) AS [Nächster runder Geburtstag]
FROM Adresse TABLESAMPLE (10 PERCENT)
WHERE MONTH(GETDATE()) = MONTH(Geburtsdatum)
ORDER BY [Geburtstag dieses Jahr];